<a href="https://colab.research.google.com/github/imanurag2311/Season_of_AI/blob/main/Week_6_first_GAN_simple_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup & Importing libraries**

In [ ]:
# Install necessary libraries if not installed
!pip install torch torchvision tensorboard

In [ ]:
# Importing libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
# to print to tensorboard

# **Defining the Discriminator Classes**

In [ ]:
# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)


# **Define The Generator Class**

In [ ]:
# Define Generator
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize outputs to [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)

# **Setting Hyperparameters**

In [ ]:
# Hyperparameters
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28  # 784 for flattened 28x28 MNIST images
batch_size = 32
num_epochs = 50

# **Initialization of models**

In [ ]:
# Initialize models and optimizers
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)


# Optimizers
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)


# Loss function
criterion = nn.BCELoss()

# **Data Loading and Preprocessing**

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

dataset = datasets.MNIST(root="dataset/", transform=transform, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 38.2MB/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.26MB/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:00<00:00, 9.60MB/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 11.6MB/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



# **Training Loop**

In [10]:
# Set up TensorBoard for visualization
writer_fake = SummaryWriter(f"logs/fake")
writer_real = SummaryWriter(f"logs/real")
step = 0


# Training Loop
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        batch_size = real.shape[0]


        ### Train Discriminator
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()


        ### Train Generator
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()


        # Print and log progress
        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )


            # Log images to TensorBoard
            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                img_grid_real = torchvision.utils.make_grid(data, normalize=True)

                writer_fake.add_image("Mnist Fake Images", img_grid_fake, global_step=step)
                writer_real.add_image("Mnist Real Images", img_grid_real, global_step=step)
                step += 1


Epoch [0/50] Batch 0/1875                       Loss D: 0.6835, loss G: 0.9058
Epoch [1/50] Batch 0/1875                       Loss D: 0.6426, loss G: 1.2775
Epoch [2/50] Batch 0/1875                       Loss D: 0.5720, loss G: 1.3895
Epoch [3/50] Batch 0/1875                       Loss D: 0.4935, loss G: 1.2404
Epoch [4/50] Batch 0/1875                       Loss D: 0.4857, loss G: 1.0658
Epoch [5/50] Batch 0/1875                       Loss D: 0.6710, loss G: 0.9798
Epoch [6/50] Batch 0/1875                       Loss D: 0.6849, loss G: 0.9755
Epoch [7/50] Batch 0/1875                       Loss D: 0.6172, loss G: 0.8694
Epoch [8/50] Batch 0/1875                       Loss D: 0.4951, loss G: 1.3441
Epoch [9/50] Batch 0/1875                       Loss D: 0.6666, loss G: 0.9997
Epoch [10/50] Batch 0/1875                       Loss D: 0.7536, loss G: 0.6442
Epoch [11/50] Batch 0/1875                       Loss D: 0.5310, loss G: 1.0697
Epoch [12/50] Batch 0/1875                       L